In [ ]:
%pip install --upgrade --quiet  lark qdrant-client

In [1]:
from langchain_community.vectorstores import Qdrant
from langchain_core.documents import Document
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.docstore.document import Document
from typing import List, Dict
from langchain.chains import RetrievalQA
import os

HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.1,api_key= os.getenv('llm_api_1'))


# Function to load and chunk data from a folder
def load_and_chunk_data(data_path):
    docs = []
    # Load all .txt files from the specified folder
    for filename in os.listdir(data_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(data_path, filename)
            loader = TextLoader(file_path, encoding='utf-8')
            docs.extend(loader.load())

    # Define headers to split on for Markdown splitting
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    # Initialize Markdown splitter
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=False
    )

    chunk_size = 512
    chunk_overlap = 0

    # Initialize character-based splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    chunked_docs = []

    # Process each document
    for doc in docs:
        # Split document by Markdown headers
        md_header_splits = markdown_splitter.split_text(doc.page_content)

        # Further split the Markdown header splits into smaller chunks
        chunked_docs.extend(text_splitter.split_documents(md_header_splits))

    return chunked_docs


data_path = '/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/thongtintuyensinh'
chunked_data = load_and_chunk_data(data_path)


vectorstore = Qdrant.from_documents(
    chunked_data,
    HF_EMBEDDING,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="tintuyensinh",
)

/home/justtuananh/miniconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/justtuananh/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/justtuananh/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`

In [5]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 20})
results = retriever.invoke("năm 2024 học viện lấy bao nhiêu chỉ tiêu nữ")
for result in results: 
    print(result.page_content)

## 4\.Chỉ tiêu tuyển sinh  
Thực hiện theo Quyết định của Bộ Quốc phòng giao. Chỉ tiêu tuyển sinh được xác định theo giới tính (thí sinh nam, thí sinh nữ) và vùng, miền (miền Bắc được tính từ Quảng Bình trở ra, miền Nam được tính từ Quảng Trị trở vào). Chỉ tiêu tuyển sinh năm 2024 sẽ được Học viện thông báo chi tiết sau khi Bộ Quốc phòng giao chỉ tiêu chính thức. Các đối tượng tuyển sinh gồm:
Đối tượng tuyển sinh: Đào tạo kỹ sư quân sự (Chỉ huy, quản lý kỹ thuật)  
Mã trường: KQH
Mã ngành: 7860220
\- Thí sinh đạt học lực Giỏi, hạnh kiểm Tốt năm lớp 10, lớp 11 và học kỳ 1 lớp 12 và có kết quả điểm bài thi đánh giá năng lực quốc tế SAT từ 1.068 điểm trở lên hoặc ACT từ 18 điểm trở lên (chứng chỉ còn thời hạn sử dụng đến thời điểm xét tuyển), tốt nghiệp THPT năm 2024.
Một là, Học viện là môi trường tốt để các em phát huy được năng lực bản thân, được học tập để khám phá và làm chủ các công nghệ kỹ thuật tiên tiến được ứng dụng trong quân sự như: Trí tuệ nhân tạo, khoa học máy tính, công ng

### Retrieval

In [ ]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
# add self-query
from langchain.chains.query_constructor.base import (
    get_query_constructor_prompt,
    load_query_constructor_runnable,
    AttributeInfo)
# Cấu hình thông tin metadata
metadata_field_info = [
    AttributeInfo(
        name="Header 1",
        description="""
Đây là tiêu đề cấp cao nhất, thường được sử dụng để giới thiệu tổng quan về chủ đề chính. 
Khi lọc dữ liệu, nếu truy vấn chứa lỗi chính tả hoặc cách viết khác của tên này, bạn nên lọc dựa trên giá trị của trường này thay vì dựa vào tên trong truy vấn.
 Phải kết hợp bộ lọc tiêu đề cấp 1 với các bộ lọc phụ đề sử dụng toán tử AND. 
 Luôn sử dụng một hoặc nhiều bộ lọc like và sử dụng toán tử OR để kiểm tra tất cả các trường khác. 
 Nếu giá trị của trường này chứa từ hoặc cụm từ tương tự như trong truy vấn, hãy lọc theo chuỗi chính xác từ giá trị thay vì từ truy vấn.
""",
        type="string",
    ),
    AttributeInfo(
        name="Header 2",
        description="""
Đây là tiêu đề cấp hai, được sử dụng để chia nhỏ nội dung thành các phần nhỏ hơn. 
Luôn luôn sử dụng một hoặc nhiều bộ lọc like và sử dụng toán tử OR để kiểm tra tất cả các trường khác.
Nếu giá trị của trường này chứa từ hoặc cụm từ tương tự như trong truy vấn, hãy lọc theo chuỗi chính xác từ giá trị thay vì từ truy vấn.
Không bao giờ lọc trường này dựa trên giá trị của trường 'Header 1'.
""",
        type="string",
    ),
    AttributeInfo(
        name="Header 3",
        description="""
Đây là tiêu đề cấp ba, cung cấp thông tin chi tiết hơn. 
Luôn luôn sử dụng một hoặc nhiều bộ lọc like và sử dụng toán tử OR để kiểm tra tất cả các trường khác. 
Nếu giá trị của trường này chứa từ hoặc cụm từ tương tự như trong truy vấn, hãy lọc theo chuỗi chính xác từ giá trị thay vì từ truy vấn. 
Không bao giờ lọc trường này dựa trên giá trị của trường 'Header 1'
""",
        type="string",
    )
]


document_content_description = "Tài liệu nói về thông tin tuyển sinh của học viện kỹ thuật quân sự trong năm nay và các năm gần đây."

# Tạo SelfQueryRetriever
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True, enable_limit=True,
)


In [ ]:
examples = [
    (
        " Chỉ tiêu tuyển sinh của học viện năm 2024",
        {"query": "chỉ tiêu tuyển sinh, học viên năm 2024",
         "filter": 'and(like("Header 1", "THÔNG TIN KỲ TUYỂN SINH CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ NĂM 2024"), or(like("Header 2", "Chỉ tiêu tuyển sinh"), '
                   'like("Header 3", "Chỉ tiêu tuyển sinh")))'}
    ),
    (
        "Các phương thức xét tuyển tại học viện kỹ thuật quân sự",
        {"query": "phương thức xét tuyển, học viện kỹ thuật quân sự",
         "filter": 'and(like("Header 1", "THÔNG TIN KỲ TUYỂN SINH CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ NĂM 2024"), or(like("Header 2", "Tổ chức xét tuyển"), '
                   'like("Header 3", "Phương thức 1: Xét tuyển thẳng**")))'}
    ),
    (
        "Điểm trúng tuyển năm 2023 của học viện",
        {"query": "trúng tuyển, 2023",
         "filter": 'and(like("Header 1", "Thông tin tuyển sinh năm 2021, 2022, 2023 của học viện kỹ thuật quân sự"), or(like("Header 2", "4.2. Điểm trúng tuyển năm 2021, 2022, 2023**"), '
                   'like("Header 3", "4.2. Điểm trúng tuyển năm 2021, 2022, 2023**")))'}
    )
]

In [ ]:
examples = [
    (
        "Admission quota of the academy for 2024",
        {
            "query": "admission quota, academy, 2024",
            "filter": 'and(like("Header 1", "ADMISSION INFORMATION OF MILITARY TECHNICAL ACADEMY IN 2024"), '
                      'or(like("Header 2", "Admission quota"), like("Header 3", "Admission quota")))'
        }
    ),
    (
        "Admission methods at the Military Technical Academy",
        {
            "query": "admission methods, military technical academy",
            "filter": 'and(like("Header 1", "ADMISSION INFORMATION OF MILITARY TECHNICAL ACADEMY IN 2024"), '
                      'or(like("Header 2", "Admission process"), like("Header 3", "Method 1: Direct admission**")))'
        }
    ),
    (
        "Admission scores for 2023 of the academy",
        {
            "query": "admission scores, 2023",
            "filter": 'and(like("Header 1", "Admission information of Military Technical Academy for 2021, 2022, 2023"), '
                      'or(like("Header 2", "4.2. Admission scores for 2021, 2022, 2023**"), '
                      'like("Header 3", "4.2. Admission scores for 2021, 2022, 2023**")))'
        }
    )
]


In [ ]:
prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
    examples=examples
)
query = "Tôi vào học viện rôi thì phải đóng bao nhiêu tiền học phí vậy?"
print(prompt.format(query="{query}"))

In [ ]:
chain = load_query_constructor_runnable(
    llm=llm,
    attribute_info=metadata_field_info,
    document_contents=document_content_description,
    examples=examples,
    fix_invalid=True
)

In [ ]:
query = "chỉ tiêu tuyển sinh của học viện năm 2024"
chain.invoke(({"query": query}))

In [ ]:
from langchain.retrievers import SelfQueryRetriever

retriever = SelfQueryRetriever(
    query_constructor=chain, vectorstore=vectorstore, verbose=True
)

In [ ]:
results = retriever.invoke(
    "Chỉ tiêu tuyển sinh của học viện năm 2024"
)
for res in results:
    print(res.page_content)
    print("\n" + "-" * 20 + "\n")

In [ ]:
sq_retriever = SelfQueryRetriever(
    query_constructor=chain,
    vectorstore=vectorstore,
    verbose=True,
)
sq_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=sq_retriever,
    return_source_documents=True
)

In [ ]:
query = "chỉ tiêu tuyển sinh của học viện năm 2023"
response = sq_qa({"query": query})
print(response)

In [8]:
sq_retriever = SelfQueryRetriever(
    query_constructor=chain,
    vectorstore=vectorstore,
    verbose=True,
)

In [9]:
# không add thêm ví dụ vào 
sq_retriever.invoke("học khối nào thi được vào học viện")

ValueError: Invalid path